# Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a
rectangular grid (of size height x width ) where each cell has a random integer value between
0 and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right
corner of the grid as fast as possible.
You can implement one of the two (or both, for no extra point) game modes:
- The time spent on a cell is the number on this cell
- The time spent on a cell is the absolute of the difference between the previous cell the
agent was on and the current cell it is on.

In order to solve this problem, you will provide 3 algorithms:
- A first baseline of your choosing. E.g. it can be any search algorithm, or an algorithm
using heuristics. It doesn’t have to perform fast or well, but should be better than
random movements.
- Dijkstra's algorithm
- Ant colony optimization algorithm

You should describe the algorithms and compare them. Are they always solving the problem?
How long do they take depending on the size of the maze?


# Environment

In [30]:
import numpy as np
import pandas as pd

class GridEnvironment:
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns
        
        # Create a matrix
        self.grid = np.random.randint(0,9,(rows,columns))

    def printData(self):
        print(self.grid)
        
    ## A function to check if the point is not out of the grid    
    def isValid(self, row, column):
        if (row<0 or row>=self.rows or column<0 or column>=self.columns):
            return False
        else: return True
    
    def stupidAlgorithm(self):
        i = 0
        j = 0
        cost = 0
        ## Last Value to subtract from final cost
        lastValue = 0
        path = []

        ## While it haven't reached the lower right corner
        while (not(i==self.rows-1 and j==self.columns-1)):
            
                
            ## Add to a path list    
            path.append([i, j])
            
        
            ## Empty list of values
            minList = []
            
            ## Points next to the current
            point3 = [i+1, j]
            point4 = [i, j+1]    
            
            ## If there is a point below or to the right, add its value to the list
            if (self.isValid(point3[0], point3[1])):
                minList.append([self.grid[point3[0]][point3[1]], point3[0], point3[1]])
        
            if (self.isValid(point4[0], point4[1])):
                minList.append([self.grid[point4[0]][point4[1]], point4[0], point4[1]])
        
    
            ## Find the lowest value of nearby points
            # Lambda expression taken from stackoverflow
            minValue = min(minList, key=lambda x: x[0])
            lastValue = minValue[0]
            cost = minValue[0] + cost
            
            i = minValue[1]
            j = minValue[2]
            
        return cost
       
    
    
    def dijkstra(self):
        
        ## Create visited and unvisited vertices list
        unvisitedVertices = []
        visitedVertices = []
        ## Create shortest Paths list (i, j, shortestPath)
        shortestPaths = [[0,0,0]]
        for i in range(self.rows):
            for j in range(self.columns):
                unvisitedVertices.append([i, j])
                if (not(i==0 and j==0)):
                    ## Add infinity to all indices except 0 0
                    shortestPaths.append([i, j, float("inf")])
        
        ## Create a copy of shortestPaths for being able to remove items once checked
        tempShortestPaths = shortestPaths.copy()
        cost = 0
        
        while (unvisitedVertices):
            ## Find out the minimum shortest path value of unvisited vertices
            currentVertex = [min(tempShortestPaths, key=lambda x: x[2])[0],min(tempShortestPaths, key=lambda x: x[2])[1]]
            currentVertexStartDistance = min(tempShortestPaths, key=lambda x: x[2])[2]
            ## Calculate all 4 neighbours for the current vertix
            point1 = [currentVertex[0]+1, currentVertex[1]]
            point2 = [currentVertex[0], currentVertex[1]+1]
            point3 = [currentVertex[0]-1, currentVertex[1]]
            point4 = [currentVertex[0], currentVertex[1]-1]
            
            neighbourList = [point1, point2, point3, point4]
            
            ## If the neighbour is valid and the new start distance is lower than the previous, change it
            for neighbours in neighbourList:
                if (self.isValid(neighbours[0], neighbours[1]) and neighbours in unvisitedVertices):
                    distance = currentVertexStartDistance + self.grid[neighbours[0]][neighbours[1]]
                    for x in shortestPaths:
                        if (x[0] == neighbours[0] and x[1] == neighbours[1] and distance < x[2]):
                            x[2] = distance
                    for x in tempShortestPaths:
                        if (x[0] == neighbours[0] and x[1] == neighbours[1] and distance < x[2]):
                            x[2] = distance
                    
            ## Add to a visited vertix
            visitedVertices.append(currentVertex)
            ## Remove from unvisited
            unvisitedVertices.remove(currentVertex)
            
            
            ## Remove from a temporary shortestPaths variable for checkin unvisited lowest start paths
            tempShortestPaths.remove([currentVertex[0], currentVertex[1], currentVertexStartDistance])
            
        
        print(shortestPaths)
        print(shortestPaths[-1][-1])
        
    # Call inner class AntColonyOptimization algorithm function
    def antColonyOpt(self):
        antColonyObject = self.AntColonyOptimization(self.grid)
        antColonyObject.printMatrix()

    class AntColonyOptimization:
        def __init__(self, matrix):
            # Matrix environment for the shortest path search
            self.matrix = matrix
            self.matrixSize = np.shape(self.matrix)
            
            # Default values inspired by https://github.com/pjmattingly/ant-colony-optimization/blob/master/ant_colony.py
            self.alpha = 0.5 # 0<= is a parameter to control the influence of pheromone
            self.beta = 1.2 # 1<= is a parameter to control the influence of the distance
            
            # Pheromone value matrix, first initialized to zeroes
            self.pheromoneMatrix = np.copy(matrix)
            self.pheromoneMatrix.fill(0)
            
            self.pheromoneConstant = 1000.0 # pheromone constant Q in the algorithm https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms
            self.pheromoneEvaporationCoefficient = 0.40 # a parameter for decreasing pheromone levels
            
            
            
            
        
        
        def printMatrix(self):
            self.antRun()
        
        
        def antRun(self):
            # Run 50 ants
            # Ants path in 2d lists. [[[0, 1], [0,2]], [[1, 0], [1, 1]]...]
            # To avoid creating another inner class for ants. Functions defined further for getting and setting distances
            # and getting, adding paths to avoid confusion in the code
            # Ants distances defined in a seperate list
            ants = []
            popIndex = 0
            antsDistance = np.zeros(50)
            firstTime = True

            for i in range(0, 50):
                currentLocation = [0, 0]
                firstMove = True
                while (currentLocation != [self.matrixSize[0]-1, self.matrixSize[1]-1]):
                    print(ants)
                    if (firstTime == True):
                        if (firstMove == True):
                            import random
                            self.addToPath(ants, currentLocation, i, firstMove)
                            currentLocation = random.choice(self.nextPossibleMoves(currentLocation, ants, i, firstTime))
                            firstMove = False

                        else:
                            import random
                            try:
                                self.addToPath(ants, currentLocation, i, firstMove)
                                currentLocation = random.choice(self.nextPossibleMoves(currentLocation, ants, i, firstTime))
                            except IndexError:
                                ants.pop(popIndex)
                                print("POP")
                                np.delete(antsDistance, i)
                                i = i+1
                                popIndex = popIndex - 1
                                continue
                            self.updateDistance(self.matrix[currentLocation[0]][currentLocation[1]], antsDistance, i)
                
                i = i + 1
                popIndex = popIndex + 1
                print(antsDistance)
                #while ant location is not the end location
                    # pick a path
                    # move to the path
                    # update traversed locations and distance
                
        
        # A function for getting ant's path from the 3d ants array
        def getAntPath(self, ants, antNumber):
            path = []
            for i in range(0, len(ants[antNumber])):
                path.append(ants[antNumber][i])
            return path
        # Adding to a path to ants variable for current run of ants
        def addToPath(self, ants, path, antNumber, firstMove):
            if (firstMove == True):
                ants.append([path])
            else: ants[antNumber].append(path)
        # Updating distance in antsDistance variable
        def updateDistance(self, distance, antsDistance, antNumber):
            antsDistance[antNumber] = distance + antsDistance[antNumber]
        
        #def pickAPath(self, ants, antNumber, firstTime):
            
        
        def nextPossibleMoves(self, currentLocation, ants, antNumber, firstTime):
            point1 = [currentLocation[0] + 1, currentLocation[1]]
            point2 = [currentLocation[0], currentLocation[1] + 1]
            point3 = [currentLocation[0] - 1, currentLocation[1]]
            point4 = [currentLocation[0], currentLocation[1] - 1]
            if (firstTime == False):
                points = [point1, point2, point3, point4]
            else:
                points = [point1, point2, point3]

            possibleMoves = []
            matrixSize = np.shape(self.matrix)
            for i in points:
                if (not(i[0] < 0 or i[0] > matrixSize[0]-1 or i[1] < 0 or i[1] > matrixSize[1]-1)):
                    try:
                        if (i not in ants[antNumber]):
                            possibleMoves.append(i)
                    except IndexError:
                        possibleMoves.append(i)

            
            return possibleMoves
        
        
        
        
        
    
    

In [43]:
test = GridEnvironment(2, 10)
test.printData()
print(test.stupidAlgorithm())
test.dijkstra()

[[2 0 6 1 5 7 7 4 1 5]
 [6 4 7 4 3 7 4 7 0 5]]
41
[[0, 0, 0], [0, 1, 0], [0, 2, 6], [0, 3, 7], [0, 4, 12], [0, 5, 19], [0, 6, 26], [0, 7, 30], [0, 8, 31], [0, 9, 36], [1, 0, 6], [1, 1, 4], [1, 2, 11], [1, 3, 11], [1, 4, 14], [1, 5, 21], [1, 6, 25], [1, 7, 32], [1, 8, 31], [1, 9, 36]]
36
